In [1]:
!pip install scikeras
#!pip install --upgrade scikeras

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
path_to_csv = '/content/gdrive/MyDrive/Sentiment (2).csv'

In [4]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import re
from keras.layers import LSTM, Embedding, Dense
from keras.models import Sequential

from sklearn.preprocessing import LabelEncoder

data = pd.read_csv(path_to_csv)
# Keeping only the neccessary columns
data = data[['text','sentiment']]

data['text'] = data['text'].apply(lambda x: x.lower())
#data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))

for idx, row in data.iterrows():
    row[0] = row[0].replace('rt', ' ')

max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)

X = pad_sequences(X)

embed_dim = 128
lstm_out = 196
def createmodel():
    model = Sequential()
    model.add(Embedding(max_fatures, embed_dim, input_length=X.shape[1]))
    model.add(LSTM(units=lstm_out))  # Ensure to remove any non-default configurations
    model.add(Dense(3, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
#print(model.summary())

labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data['sentiment'])
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42)

batch_size = 32
model = createmodel()
model.fit(X_train, Y_train, epochs = 1, batch_size=batch_size, verbose = 2)
score,acc = model.evaluate(X_test,Y_test,verbose=2,batch_size=batch_size)
print(score)
print(acc)
print(model.metrics_names)


291/291 - 20s - loss: 0.8183 - accuracy: 0.6524 - 20s/epoch - 68ms/step
144/144 - 1s - loss: 0.7573 - accuracy: 0.6785 - 1s/epoch - 8ms/step
0.7572862505912781
0.6784622073173523
['loss', 'accuracy']


In [5]:
model.save('sentimentalAnalysis')
from keras.models import load_model
model = load_model('sentimentalAnalysis')

In [6]:
from scikeras.wrappers import KerasClassifier
import numpy as np
sentence = ['A lot of good things are happening. We are respected again throughout the world, and thats a great thing.@realDonaldTrump']
sentence = tokenizer.texts_to_sequences(sentence)
sentence = pad_sequences (sentence, maxlen=28, dtype='int32', value=0)
#sentiment = model.predict_classes(sentence,batch_size=1,verbose = 2) [0]
#sentiment = (model.predict(sentence) > 0.5).astype("int32")
#sentiment = model.argmax(model.predict(sentence), axis=-1)
predict_x=model.predict(X_test)
sentiment=np.argmax(predict_x,axis=1)
print(sentiment)
if sentiment.any() == 0:
  print("Neural")
elif sentiment.any() < 0:
  print("Negative")
elif sentiment.any() > 0:
  print("Positive")
else:
  print("Cannot be determined")

144/144 [==============================] - 1s 3ms/step
[0 2 2 ... 0 0 0]
Positive


In [7]:
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV
#from keras.wrappers.scikit_learn import KerasClassifier
model = KerasClassifier(model=createmodel,verbose=2)
batch_size = [10, 20, 40]
epochs = [1, 2]
param_grid = {'batch_size':batch_size, 'epochs':epochs}
grid = GridSearchCV(estimator=model, param_grid=param_grid)
grid_result = grid.fit(X_train, Y_train)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

744/744 - 13s - loss: 0.8189 - accuracy: 0.6560 - 13s/epoch - 18ms/step
186/186 - 1s - 702ms/epoch - 4ms/step
744/744 - 12s - loss: 0.8139 - accuracy: 0.6591 - 12s/epoch - 16ms/step
186/186 - 1s - 698ms/epoch - 4ms/step
744/744 - 15s - loss: 0.8160 - accuracy: 0.6496 - 15s/epoch - 20ms/step
186/186 - 1s - 746ms/epoch - 4ms/step
744/744 - 15s - loss: 0.8228 - accuracy: 0.6455 - 15s/epoch - 20ms/step
186/186 - 1s - 741ms/epoch - 4ms/step
744/744 - 13s - loss: 0.8175 - accuracy: 0.6507 - 13s/epoch - 17ms/step
186/186 - 1s - 736ms/epoch - 4ms/step
Epoch 1/2
744/744 - 12s - loss: 0.8178 - accuracy: 0.6552 - 12s/epoch - 16ms/step
Epoch 2/2
744/744 - 4s - loss: 0.6754 - accuracy: 0.7151 - 4s/epoch - 5ms/step
186/186 - 1s - 721ms/epoch - 4ms/step
Epoch 1/2
744/744 - 14s - loss: 0.8163 - accuracy: 0.6531 - 14s/epoch - 19ms/step
Epoch 2/2
744/744 - 4s - loss: 0.6741 - accuracy: 0.7164 - 4s/epoch - 6ms/step
186/186 - 1s - 696ms/epoch - 4ms/step
Epoch 1/2
744/744 - 14s - loss: 0.8148 - accuracy: 0